# Preparation

### What are the parameters for LTE throughput calculation? 

The parameters for LTE throughput calculation are the Signal to Noise Ratio (SNR) and the available bandwidth.

### Derive an equation to calculate throughput.

Shannon's equation describes the maximum rate at which information can be correctly received from a channel with a given bandwidth and SNR

In [1]:
const shannonsEquation = (snr: number, bandwidth: number) => 
    bandwidth * Math.log2(1 + snr)

However, this just is the bound and relies on a few assumptions, like infinite codewords, a random codebook and an AWGN channel. 

To calculate the possible throughput for LTE we have to multiply the available bandwidth with the efficiency of the chosen modulation and code rate. These values are chosen based on the CQI according to this table from the specification:

# ![A table](another-figure.png)

In [2]:
// Values from the table above
const efficiencyValues = [
  0, 0.1523, 0.2344, 0.377, 0.6016, 0.877, 1.1758, 1.4766, 1.9141, 2.4063,
  2.7305, 3.3223, 3.9023, 4.5234, 5.1152, 5.5547,
];

const getEfficiency = (cqi: number) => {
  const efficiency = efficiencyValues[cqi];
  if (efficiency === undefined) {
    throw new Error("CQI must be an integer between 0 and 15");
  }
  return efficiency;
};

NOTE: There are also other tables, but we choose this one.

To index the table we need the CQI value. The CQI is based on the SNR, we use the the function from the exercise sheet to convert them.

# ![A table](that-figure.png)

In [3]:
const snrToCqi = (snrDb: number) => {
    const cqi = snrDb*0.525+4.5;
    const clampedCqi = Math.max(0, Math.min(15, cqi));
    const clampedAndRoundedCqi = Math.round(clampedCqi)
    return clampedAndRoundedCqi
}

When we combine these two, we can calculate the throughput in based on the SNR and the bandwidth for LTE

In [4]:
const lteThroughput = (snrDb: number, bandwidth: number)  => 
    bandwidth * getEfficiency(snrToCqi(snrDb))

console.log(`The bandwidth for 20mHz and a SNR of 7dB is ${lteThroughput(7, 20e6)} bits per second`)

The bandwidth for 20mHz and a SNR of 7dB is 38282000 bits per second


If we compare that function with the channel capacity we can see that it is always slightly lower:

In [5]:
import { plotFunctions } from "./helper.ts"

const dbToFactor = (db: number) => 
    Math.pow(10, db/10)

const bandwidth = 20e6

await plotFunctions({
  from: -15,
  to: 25,
  step: 0.2,
  xName: "SNR (dB)",
  yName: "Bits per Second",
  colorName: "__color",
  fns: [
    ["Channel capacity", (snrDb: number) => shannonsEquation(dbToFactor(snrDb), bandwidth)],
    ["LTE throughput", (snrDb: number) => lteThroughput(snrDb, bandwidth)]
  ]
})

SNR (dB) vs Bits per Second Channel capacity LTE throughput 0 20,000,000 40,000,000 60,000,000 80,000,000 100,000,000 120,000,000 140,000,000 160,000,000 ↑ Bits per Second −15 −10 −5 0 5 10 15 20 SNR (dB) → <path stroke="#4269d0" d="M40,378.093L42.915,378.004L45.829,377.912L48.744,377.815L51.658,377.714L54.573,377.609L57.487,377.498L60.402,377.383L63.317,377.262L66.231,377.136L69.146,377.004L72.06,376.867L74.975,376.723L77.889,376.573L80.804,376.416L83.719,376.252L86.633,376.081L89.548,375.902L92.462,375.716L95.377,375.521L98.291,375.318L101.206,375.106L104.121,374.884L107.035,374.654L109.95,374.413L112.864,374.162L115.779,373.9L118.693,373.627L121.608,373.342L124.523,373.046L127.437,372.736L130.352,372.415L133.266,372.079L136.181,371.73L139.095,371.367L142.01,370.988L144.925,370.595L147.839,370.185L150.754,369.759L153.668,369.316L156.583,368.856L159.497,368.378L162.412,367.881L165.327,367.365L168.241,366.829L171.156,366.273L174.07,365.696L176.985,365.098L179.899,364.477L182.814,363.835L185.729,363.169L188.643,362.479L191.558,361.765L194.472,361.026L197.387,360.262L200.302,359.472L203.216,358.655L206.131,357.812L209.045,356.941L211.96,356.042L214.874,355.114L217.789,354.158L220.704,353.172L223.618,352.157L226.533,351.111L229.447,350.036L232.362,348.929L235.276,347.791L238.191,346.622L241.106,345.421L244.02,344.188L246.935,342.923L249.849,341.626L252.764,340.297L255.678,338.935L258.593,337.541L261.508,336.114L264.422,334.655L267.337,333.163L270.251,331.639L273.166,330.083L276.08,328.495L278.995,326.875L281.91,325.223L284.824,323.54L287.739,321.826L290.653,320.081L293.568,318.306L296.482,316.5L299.397,314.665L302.312,312.8L305.226,310.907L308.141,308.985L311.055,307.035L313.97,305.057L316.884,303.053L319.799,301.022L322.714,298.965L325.628,296.883L328.543,294.776L331.457,292.645L334.372,290.49L337.286,288.312L340.201,286.111L343.116,283.889L346.03,281.644L348.945,279.38L351.859,277.094L354.774,274.789L357.688,272.465L360.603,270.123L363.518,267.762L366.432,265.384L369.347,262.989L372.261,260.578L375.176,258.15L378.09,255.708L381.005,253.25L383.92,250.778L386.834,248.293L389.749,245.794L392.663,243.282L395.578,240.758L398.492,238.221L401.407,235.673L404.322,233.114L407.236,230.544L410.151,227.964L413.065,225.374L415.98,222.775L418.894,220.166L421.809,217.548L424.724,214.922L427.638,212.287L430.553,209.645L433.467,206.995L436.382,204.338L439.296,201.674L442.211,199.003L445.126,196.326L448.04,193.643L450.955,190.954L453.869,188.259L456.784,185.558L459.698,182.853L462.613,180.142L465.528,177.427L468.442,174.707L471.357,171.983L474.271,169.254L477.186,166.522L480.101,163.785L483.015,161.045L485.93,158.302L488.844,155.555L491.759,152.805L494.673,150.051L497.588,147.295L500.503,144.536L503.417,141.774L506.332,139.009L509.246,136.242L512.161,133.473L515.075,130.701L517.99,127.927L520.905,125.151L523.819,122.374L526.734,119.594L529.648,116.812L532.563,114.029L535.477,111.244L538.392,108.457L541.307,105.669L544.221,102.879L547.136,100.088L550.05,97.296L552.965,94.502L555.879,91.707L558.794,88.911L561.709,86.114L564.623,83.316L567.538,80.517L570.452,77.716L573.367,74.915L576.281,72.113L579.196,69.31L582.111,66.507L585.025,63.702L587.94,60.897L590.854,58.091L593.769,55.285L596.683,52.478L599.598,49.67L602.513,46.861L605.427,44.052L608.342,41.243L611.256,38.433L614.171,35.622L617.085,32.811L620,30"> <path stroke="#efb118" d="M40,380L42.915,380L45.829,380L48.744,380L51.658,380L54.573,380L57.487,380L60.402,380L63.317,380L66.231,380L69.146,380L72.06,380L74.975,380L77.889,380L80.804,380L83.719,380L86.633,380L89.548,380L92.462,380L95.377,380L98.291,380L101.206,380L104.121,380L107.035,380L109.95,380L112.864,380L115.779,380L118.693,380L121.608,380L124.523,380L127.437,380L130.352,380L133.266,380L136.181,380L139.095,380L142.01,380L144.925,380L147.839,373.533L150.754,373.533L153.668,373.533L156.583,373.533L159.497,373.533L162.412,373.533L165.327,373.533L168.241,373.533L171.156,373.533L174.07,373.533L176.985,370.047L179.899,370.047L182.814,370.047L185.

# LTE throughput at the link layer

We measured the bandwidth and SNR five times on the same Smartphone and got the following results:

In [6]:
const measurementsExerciseTwo = [
    {bandwidth: 20, snr: 8.4},
    {bandwidth: 10, snr: 11.8},
    {bandwidth: 20, snr: 8.6},
    {bandwidth: 10, snr: 12.4},
    {bandwidth: 20, snr: 7.4},
]

It is interesting that the 

# Interference in LTE

In [7]:
const measurementsExerciseThree = [
    {rsrp: -103, snr: 7.6, bandwidth: 20, neighbours: [
        {id: 51, rsrp: -112},
        {id: 53, rsrp: -108},
        {id: 485, rsrp: -111},
    ]},
    {id: 262, rsrp: -100, snr: 8.2, bandwidth: 20, neighbours: [
        {id: 51, rsrp: -113},
        {id: 53, rsrp: -115},
        {id: 485, rsrp: -113},
        {id: 331, rsrp: -114},
    ]},
    {id: 262, rsrp: -96, snr: 9.8, bandwidth: 20, neighbours: [
        {id: 326, rsrp: -108},
        {id: 155, rsrp: -109},
        {id: 429, rsrp: -103},
    ]},
    {id: 262, rsrp: -99, snr: 8.4, bandwidth: 20, neighbours: [
        {id: 51, rsrp: -109},
        {id: 485, rsrp: -113},
    ]},
    {id: 262, rsrp: -100, snr: 7.2, bandwidth: 20, neighbours: [
        {id: 51, rsrp: -113},
        {id: 53, rsrp: -112},
        {id: 253, rsrp: -113},
    ]},
]